# Machine Learning Pipeline with Tensorflow Extended

#### Import libraries

In [ ]:
!pip install tfx

In [29]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec
from tfx.utils.dsl_utils import external_input
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx import components
import tensorflow_model_analysis as tfma

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [30]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.3.2
TFX version: 0.27.0


In [31]:
tfx.__path__[0]

'/opt/conda/lib/python3.7/site-packages/tfx'

In [32]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the Census data example.
_data_root = "gs://crazy-hippo-01/tfx/binary_classification/raw/"

# This is the path where your model will be pushed for serving.
_serving_model_dir = 'gs://crazy-hippo-01/tfx/binary_classification/serving_model/'

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

#### Set Context to Interactive

In [33]:
context = InteractiveContext()

## 1. Load data 

#### Loading CSV files 

ExampleGen provides data to components that make use of the TensorFlow Data Validation library, such as SchemaGen, StatisticsGen, and Example Validator. It also provides data to Transform, which makes use of the TensorFlow Transform library, and ultimately to deployment targets during inference.

The ExampleGen component is usually at the start of a TFX pipeline. It will:

* Split data into training and evaluation sets (by default, 2/3 training + 1/3 eval)
* Convert data into the tf.Example format
* Copy data into the _tfx_root directory for other components to access

In [34]:
example_gen = CsvExampleGen(input_base=_data_root)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data gs://crazy-hippo-01/tfx/binary_classification/raw/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:3424514,xor_checksum:1587724005,sum_checksum:1587724005"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [35]:
%%skip_for_export

artifact = example_gen.outputs['examples'].get()[0]
print(artifact.type)
print(artifact.split_names)
print(artifact.uri)
print(artifact.version)

<class 'tfx.types.standard_artifacts.Examples'>
["train", "eval"]
/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/CsvExampleGen/examples/1
0
This cell will be skipped during export to pipeline.


In [36]:
%%skip_for_export

# Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 39
      }
    }
  }
  feature {
    key: "capital_gain"
    value {
      int64_list {
        value: 2174
      }
    }
  }
  feature {
    key: "capital_loss"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "education"
    value {
      bytes_list {
        value: "Bachelors"
      }
    }
  }
  feature {
    key: "education_num"
    value {
      int64_list {
        value: 13
      }
    }
  }
  feature {
    key: "fnlwgt"
    value {
      int64_list {
        value: 77516
      }
    }
  }
  feature {
    key: "gender"
    value {
      bytes_list {
        value: "Male"
      }
    }
  }
  feature {
    key: "hours_per_week"
    value {
      int64_list {
        value: 40
      }
    }
  }
  feature {
    key: "income_bracket"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "marital_status"
    value {
      bytes_lis

## 2. Explore and Understand the data

#### Use StatisticsGen to better understand training and evaluation data. 

The StatisticsGen TFX pipeline component generates features statistics 
over both training and serving data, which can be used by other pipeline 
components. StatisticsGen uses Beam to scale to large datasets.

* Consumes: datasets created by an ExampleGen pipeline component.
* Emits: Dataset statistics.

In [37]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/StatisticsGen/statistics/2/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [38]:
%%skip_for_export

context.show(statistics_gen.outputs['statistics'])

This cell will be skipped during export to pipeline.


## 3. Apply a Schema on top of your data

Some TFX components use a description of your input data called a schema. The schema is an instance of schema.proto. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. A SchemaGen pipeline component will automatically generate a schema by inferring types, categories, and ranges from the training data.

* Consumes: statistics from a StatisticsGen component
* Emits: Data schema proto

In [39]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False
)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [40]:
%%skip_for_export

schema_gen.outputs['schema']

Channel(
    type_name: Schema
    artifacts: [Artifact(artifact: id: 3
type_id: 9
uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/SchemaGen/schema/3"
custom_properties {
  key: "name"
  value {
    string_value: "schema"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "SchemaGen"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 9
name: "Schema"
)]
    additional_properties: {}
    additional_custom_properties: {}
)

This cell will be skipped during export to pipeline.


In [41]:
%%skip_for_export

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,single,-
'capital_gain',INT,required,single,-
'capital_loss',INT,required,single,-
'education',STRING,required,single,'education'
'education_num',INT,required,single,-
'fnlwgt',INT,required,single,-
'gender',STRING,required,single,'gender'
'hours_per_week',INT,required,single,-
'income_bracket',INT,required,single,-


/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'education',"'10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th', 'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad', 'Masters', 'Preschool', 'Prof-school', 'Some-college'"
'gender',"'Female', 'Male'"
'marital_status',"'Divorced', 'Married-AF-spouse', 'Married-civ-spouse', 'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'"
'native_country',"'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba', 'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France', 'Germany', 'Greece', 'Guatemala', 'Haiti', 'Honduras', 'Hong', 'Hungary', 'India', 'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico', 'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan', 'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam', 'Yugoslavia', 'Holand-Netherlands'"
'occupation',"'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial', 'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct', 'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'"
'race',"'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White'"
'relationship',"'Husband', 'Not-in-family', 'Other-relative', 'Own-child', 'Unmarried', 'Wife'"
'workclass',"'Federal-gov', 'Local-gov', 'Never-worked', 'Private', 'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'"


This cell will be skipped during export to pipeline.


## 4. Perform Data Validation

The ExampleValidator pipeline component identifies anomalies in training and serving data. It can detect different classes of anomalies in the data. For example it can:

* perform validity checks by comparing data statistics against a schema that codifies expectations of the user
* detect training-serving skew by comparing training and serving data.
* detect data drift by looking at a series of data.

The ExampleValidator pipeline component identifies any anomalies in the example data by comparing data statistics computed by the StatisticsGen pipeline component against a schema. The inferred schema codifies properties which the input data is expected to satisfy, and can be modified by the developer.

* Consumes: A schema from a SchemaGen component, and statistics from a StatisticsGen component.
* Emits: Validation results

In [42]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/ExampleValidator/anomalies/4/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/ExampleValidator/anomalies/4/eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [43]:
%%skip_for_export

context.show(example_validator.outputs['anomalies'])

This cell will be skipped during export to pipeline.


# 5. Data Preparation & Feature Engineering

The Transform TFX pipeline component performs feature engineering on tf.Examples emitted from an ExampleGen component, using a data schema created by a SchemaGen component, and emits a SavedModel. When executed, the SavedModel will accept tf.Examples emitted from an ExampleGen component and emit the transformed feature data.

* Consumes: tf.Examples from an ExampleGen component, and a data schema from a SchemaGen component.
* Emits: A SavedModel to a Trainer component

In [44]:
_binary_constants_module_file = 'binary_constants.py'

In [45]:
%%skip_for_export
%%writefile {_binary_constants_module_file}

CATEGORICAL_FEATURES = ['workclass', 'education', 'marital_status', 'occupation', 'relationship',
    'race', 'gender', 'native_country']

NUMERIC_FEATURES = [ 'age', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', 'fnlwgt']

INPUTS = ['workclass', 'education', 'marital_status', 'occupation', 'relationship',
    'race', 'gender', 'native_country', 'age', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', 'fnlwgt']

LABEL_KEY = 'income_bracket'

Overwriting binary_constants.py
This cell will be skipped during export to pipeline.


In [46]:
_binary_transform_module_file = 'binary_transform.py'

In [47]:
%%skip_for_export
%%writefile {_binary_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft
import binary_constants
import importlib

importlib.reload(binary_constants)

NUMERIC_FEATURES = binary_constants.NUMERIC_FEATURES
CATEGORICAL_FEATURES = binary_constants.CATEGORICAL_FEATURES
LABEL_KEY = binary_constants.LABEL_KEY
#VOCAB_SIZE = binary_constants.VOCAB_SIZE
#OOV_SIZE = binary_constants.OOV_SIZE
#TRANSFORMED_NAME = binary_constants.transformed_name

def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
      Args:
        inputs: map from feature keys to raw not-yet-transformed features.
      Returns:
        Map from string feature key to transformed feature operations.
    """
    outputs = inputs.copy()
    
    #Scale numeric columns to have range [0,1] - if Tensors.
    for key in NUMERIC_FEATURES:
        #outputs[key] = tft.scale_to_0_1(inputs[key])
        #outputs[transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))
        outputs[key] = tft.scale_to_0_1(_fill_in_missing(outputs[key]))
    
    # For all categorical columns except the label column, we generate a
    # vocabulary but do not modify the feature.  This vocabulary is instead
    # used in the trainer, by means of a feature column, to convert the feature
    # from a string to an integer id.
    
    for key in CATEGORICAL_FEATURES:
        outputs[key] = tft.compute_and_apply_vocabulary(
                                            _fill_in_missing(outputs[key]),
                                            #top_k=VOCAB_SIZE,
                                            #num_oov_buckets=OOV_SIZE
                                            )
    outputs[LABEL_KEY] = _fill_in_missing(outputs[LABEL_KEY])
    
    return outputs

def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
        x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
          in the second dimension.
        Returns:
        A rank 1 tensor where missing values of `x` have been filled in.
    """     
    
    if not isinstance(x, tf.sparse.SparseTensor):
        return x
    
    default_value = '' if x.dtype == tf.string else 0
    
    return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Overwriting binary_transform.py
This cell will be skipped during export to pipeline.


In [48]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=_binary_transform_module_file
)
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.
INFO:absl:binary_transform.py is already loaded.
INFO:absl:binary_transform.py is already loaded.
INFO:absl:Feature age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature capital_gain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature capital_loss has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature education has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature education_num has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fnlwgt has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature gender has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature hours_per_week has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature income_bracket has no shape. Setting to VarLenSparseTensor.
INFO:absl:

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/transform_graph/5/.temp_path/tftransform_tmp/ee1ce181e4944be7bdabc43df7a53218/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/transform_graph/5/.temp_path/tftransform_tmp/b13f40b7a560448a89a1cf0bf241b02e/saved_model.pb


INFO:absl:Feature age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature capital_gain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature capital_loss has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature education has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature education_num has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fnlwgt has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature gender has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature hours_per_week has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature income_bracket has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature marital_status has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature native_country has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature occupation has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature race has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature relationship has no

INFO:absl:Feature age has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature capital_gain has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature capital_loss has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature education has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature education_num has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature fnlwgt has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature gender has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature hours_per_week has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature income_bracket has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature marital_status has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature native_country has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature occupation has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature race has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature relationship has no

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/transform_graph/5/.temp_path/tftransform_tmp/42d18011239a44e4976cb5c0c0785552/assets
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/transform_graph/5/.temp_path/tftransform_tmp/42d18011239a44e4976cb5c0c0785552/saved_model.pb
value: "\n\013\n\tConst_3:0\022-vocab_compute_and_apply_vocabulary_vocabulary"

value: "\n\013\n\tConst_5:0\022/vocab_compute_and_apply_vocabulary_1_vocabulary"

value: "\n\013\n\tConst_7:0\022/vocab_compute_and_apply_vocabulary_2_vocabulary"

value: "\n\013\n\tConst_9:0\022/vocab_compute_and_apply_vocabulary_3_vocabulary"

value: "\n\014\n\nConst_11:0\022/vocab_compute_and_apply_vocabulary_4_v

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [49]:
%%skip_for_export

transform.outputs

{'transform_graph': Channel(
    type_name: TransformGraph
    artifacts: [Artifact(artifact: id: 5
type_id: 13
uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/transform_graph/5"
custom_properties {
  key: "name"
  value {
    string_value: "transform_graph"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Transform"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 13
name: "TransformGraph"
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'transformed_examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 6
type_id: 5
uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Transform/transformed_examples/5"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "transformed_examples"
  }
}
custom_propert

This cell will be skipped during export to pipeline.


In [52]:
%%skip_for_export

train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transform_fn', 'transformed_metadata', 'metadata']

This cell will be skipped during export to pipeline.


In [53]:
%%skip_for_export

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = tf.train.Example()
    example.ParseFromString(serialized_example)
    pp.pprint(example)

features {
  feature {
    key: "age"
    value {
      float_list {
        value: 0.301369845867157
      }
    }
  }
  feature {
    key: "capital_gain"
    value {
      float_list {
        value: 0.02174021676182747
      }
    }
  }
  feature {
    key: "capital_loss"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "education"
    value {
      int64_list {
        value: 2
      }
    }
  }
  feature {
    key: "education_num"
    value {
      float_list {
        value: 0.8000000715255737
      }
    }
  }
  feature {
    key: "fnlwgt"
    value {
      float_list {
        value: 0.044301897287368774
      }
    }
  }
  feature {
    key: "gender"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "hours_per_week"
    value {
      float_list {
        value: 0.3979591727256775
      }
    }
  }
  feature {
    key: "income_bracket"
    value {
      int64_list {
        value: 0
      }
    }
  }
 

# 6. Training the Model

The Trainer TFX pipeline component trains a TensorFlow model.

Trainer takes:

* tf.Examples used for training and eval.
* A user provided module file that defines the trainer logic.
* Protobuf definition of train args and eval args.
* (Optional) A data schema created by a SchemaGen pipeline component and optionally altered by the developer.
* (Optional) transform graph produced by an upstream Transform component.
* (Optional) pre-trained models used for scenarios such as warmstart.
* (Optional) hyperparameters, which will be passed to user module function. Details of the integration with Tuner can be found here.

In [54]:
functional_binary_trainer = 'functional_binary_trainer.py'

In [55]:
trainer = Trainer(
    module_file=os.path.abspath(functional_binary_trainer),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500) 
)

context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:/home/jupyter/notebook-demo1/tfx/functional_binary_trainer.py is already loaded.
INFO:absl:Training model.


Your model has been built....
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model/6/serving_model_dir/assets


INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model/6/serving_model_dir. ModelRun written to /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model_run/6
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 8
        type_id: 16
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [56]:
%%skip_for_export

model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')
pp.pprint(os.listdir(model_dir))

['serving_model_dir']
['variables', 'assets', 'saved_model.pb']
This cell will be skipped during export to pipeline.


In [57]:
%%skip_for_export

model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


This cell will be skipped during export to pipeline.


# 7. Evaluation of Model

The Evaluator TFX pipeline component performs deep analysis on the training results for your models, to help you understand how your model performs on subsets of your data. The Evaluator also helps you validate your exported models, ensuring that they are "good enough" to be pushed to production.
<br><br>
When validation is enabled, the Evaluator compares new models against a baseline (such as the currently serving model) to determine if they're "good enough" relative to the baseline. It does so by evaluating both models on an eval dataset and computing their performance on metrics (e.g. AUC, loss). If the new model's metrics meet developer-specified criteria relative to the baseline model (e.g. AUC is not lower), the model is "blessed" (marked as good), indicating to the Pusher that it is ok to push the model to production.

Consumes:
* An eval split from ExampleGen
* A trained model from Trainer
* A previously blessed model (if validation to be performed)
<br><br>
Emits:
* Analysis results to ML Metadata
* Validation results to ML Metadata (if validation to be performed)

In [73]:
%%skip_for_export

#eval_config = tfma.EvalConfig(
#    model_specs=[
        # Using signature 'eval' implies the use of an EvalSavedModel. To use
        # a serving model remove the signature to defaults to 'serving_default'
        # and add a label_key.
#        tfma.ModelSpec(signature_name='default_signature'),
#        tfma.ModelSpec(label_key='income_bracket'),
        #signature_name="serving_default",
#    ],
#    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
#        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
#        tfma.SlicingSpec(feature_keys=['age'])
#    ]
#)

accuracy_threshold = tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.5},
                    upper_bound={'value': 0.99}),
                )
    
metrics_specs = tfma.MetricsSpec(
                   metrics = [
                       tfma.MetricConfig(class_name='SparseCategoricalAccuracy',
                           threshold=accuracy_threshold),
                       tfma.MetricConfig(class_name='ExampleCount')])


eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='income_bracket'),
        tfma.ModelSpec(signature_name='default_signature')
    ],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['age'])
    ]
  )

This cell will be skipped during export to pipeline.


In [59]:
    metrics_specs=[
        tfma.MetricsSpec(
            
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount')
            ],
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            thresholds = {
                'accuracy': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value': 0.5})
                    ,
                    change_threshold=tfma.GenericChangeThreshold(
                       direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                       absolute={'value': -1e-10})
                )
            }
        )
    ],

In [60]:
%%skip_for_export
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
context.run(model_resolver)

INFO:absl:Running driver for Resolver.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for Resolver.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Resolver.latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
            additional_properties: {}
            additional_custom_properties: {}
        ))

This cell will be skipped during export to pipeline.


In [77]:
%%skip_for_export

# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    #baseline_model=model_resolver.outputs['model'],
    # Change threshold will be ignored if there is no baseline (first run).
    eval_config=eval_config
)
context.run(evaluator)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "income_bracket"
}
model_specs {
  signature_name: "default_signature"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "age"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.5
        }
        upper_bound {
          value: 0.99
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

INFO:absl:Using /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model/6/serving_model_dir as  model.


INFO:absl:Using /tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model/6/serving_model_dir as  model.


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.


ValueError: model_name is required when passing multiple EvalSharedModels: eval_shared_models=[EvalSharedModel(model_path='/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model/6/serving_model_dir', add_metrics_callbacks=[], include_default_metrics=True, example_weight_key=None, additional_fetches=None, model_loader=<tensorflow_model_analysis.types.ModelLoader object at 0x7f4ffb3b2820>, model_name='', model_type='tf_keras', rubber_stamp=False, is_baseline=False), EvalSharedModel(model_path='/tmp/tfx-interactive-2021-03-17T08_26_55.138083-xdpbjhom/Trainer/model/6/serving_model_dir', add_metrics_callbacks=[], include_default_metrics=True, example_weight_key=None, additional_fetches=None, model_loader=<tensorflow_model_analysis.types.ModelLoader object at 0x7f4ff8d38050>, model_name='', model_type='tf_keras', rubber_stamp=False, is_baseline=False)]

This cell will be skipped during export to pipeline.


In [ ]:
%%skip_for_export

evaluator.outputs

In [ ]:
%%skip_for_export

context.show(evaluator.outputs['evaluation'])

In [ ]:
%%skip_for_export

import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

#tfma.view.render_plot(
#    tfma_result,
#    slicing_spec=None,
    #label="V1"
#)

# Show data sliced along feature column trip_start_hour.
tfma.view.render_slicing_metrics(
    tfma_result, 
    slicing_column='age'
)

# 8. Deploy Model for Serving

The Pusher component is used to push a validated model to a deployment target during model training or re-training. Before the deployment, Pusher relies on one or more blessings from other validation components to decide whether to push the model or not.

* Evaluator blesses the model if the new trained model is "good enough" to be pushed to production.
* (Optional but recommended) InfraValidator blesses the model if the model is mechanically servable in a production environment.

A Pusher component consumes a trained model in SavedModel format, and produces the same SavedModel, along with versioning metadata.

In [ ]:
%%skip_for_export

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

In [ ]:
%%skip_for_export

pusher.outputs

In [ ]:
%%skip_for_export

push_uri = pusher.outputs.pushed_model.get()[0].uri
#push_uri = pusher.outputs.model_push.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

# 9. Pipeline

In [ ]:
_runner_type = 'beam' 
_pipeline_name = 'census_binary_%s' % _runner_type

In [ ]:
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_census_root = os.path.join(_tfx_root,'binary_census_model', 'pipeline_run')
_serving_model_dir = 'gs://crazy-hippo-01/tfx/binary_classification/serving_model/'
_data_root = os.path.join(_census_root, 'data', 'simple')
_pipeline_root = os.path.join(_census_root, 'pipelines', _pipeline_name)
_metadata_path = os.path.join(_census_root, 'metadata', _pipeline_name, 'metadata.db')

In [ ]:
#Specify components to be included in the exported pipeline.
components = [
    example_gen, statistics_gen, schema_gen, example_validator, transform,
    trainer
]

In [ ]:
%%skip_for_export

if get_ipython().magics_manager.auto_magic:
  print('Warning: %automagic is ON. Line magics specified without the % prefix '
        'will not be scrubbed during export to pipeline.')

_pipeline_export_filepath = 'export_%s.py' % _pipeline_name
context.export_to_pipeline(notebook_filepath=_notebook_filepath,
                           export_filepath=_pipeline_export_filepath,
                           runner_type=_runner_type)